In [2]:
!pip install labelImg

  Using cached labelImg-1.8.6-py2.py3-none-any.whl


In [4]:
import os
import cv2
import numpy as np
import xml.etree.ElementTree as ET
from labelImg import labelImg
from keras.models import Model
from keras.layers import Input, Flatten, Dense
from keras.utils import to_categorical
from keras.applications import VGG16
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint

In [5]:
# Define constants
NUM_CLASSES = 6
BATCH_SIZE = 16
NUM_EPOCHS = 50
LEARNING_RATE = 1e-4

In [6]:
# Define paths to data
DATA_DIR = "C:\\Users\\zigya\\OneDrive\\Desktop\\Invoice-2\\data"
IMAGE_DIR = os.path.join(DATA_DIR, "images")
ANNOT_DIR = os.path.join(DATA_DIR, "labels")
TRAIN_DIR = os.path.join(DATA_DIR, "train")
VAL_DIR = os.path.join(DATA_DIR, "val")
TEST_DIR = os.path.join(DATA_DIR, "test")

# Define class names
CLASS_NAMES = ["Company Name", "Name", "Address", "Item Description", "Item Amount", "Total Amount"]


In [7]:
# Define function to load data
def load_data(data_dir, split):
    image_dir = os.path.join(data_dir, split)
    annot_dir = os.path.join(data_dir, "labels")
    images = []
    labels = []
    for filename in os.listdir(image_dir):
        if filename.endswith(".jpg"):
            image_path = os.path.join(image_dir, filename)
            annot_path = os.path.join(annot_dir, os.path.splitext(filename)[0] + ".xml")
            tree = ET.parse(annot_path)
            root = tree.getroot()
            boxes = []
            for obj in root.iter("object"):
                class_name = obj.find("name").text
                xmin = int(obj.find("bndbox").find("xmin").text)
                ymin = int(obj.find("bndbox").find("ymin").text)
                xmax = int(obj.find("bndbox").find("xmax").text)
                ymax = int(obj.find("bndbox").find("ymax").text)
                boxes.append([xmin, ymin, xmax, ymax, CLASS_NAMES.index(class_name)])
            if boxes:
                images.append(cv2.imread(image_path))
                labels.append(np.array(boxes))
    return np.array(images), np.array(labels)

In [9]:
# Load data
X_train, y_train = load_data(DATA_DIR, "train")
X_val, y_val = load_data(DATA_DIR, "val")

In [10]:
# Load pre-trained VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(None,None,3)))

# Freeze layers in the base model
for layer in base_model.layers:
    layer.trainable = False

58889256/58889256 [==============================] - 10s 0us/step


In [18]:
# Get the output of the base model
x = base_model.output

# Flatten the output
x = Flatten()(x)

# Define the input shape for the Dense layer
input_shape = (x.shape[1],)

print(input_shape)


(None,)


In [ ]:
# Add new layers for classification
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dense(NUM_CLASSES, activation="softmax")(x)

# Create new model
model = Model(inputs=base_model.input, outputs=x)

# Compile model
sgd = SGD(lr=LEARNING_RATE, momentum=0.9, decay=0.0, nesterov=False)
model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["accuracy"])

# Create directories for training
if not os.path.exists(TRAIN_DIR):
    os.makedirs(TRAIN_DIR)
if not os.path.exists(VAL_DIR):
    os.makedirs(VAL_DIR)

# Convert labels to categorical format
y_train_cat = to_categorical(y_train[:, :, 4], num_classes=NUM_CLASSES)
y_val_cat = to_categorical(y_val[:, :, 
